#### Lunar Hopper Simulation Analysis example

This example shows the basic CITROS data analysis functions and based on the Lunar Hopper simulation with MPOPT library.



First of all, we should import all neccesary libs:

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from prettytable import PrettyTable, ALL

import json

from platform import python_version

from citros import CitrosDB


Now we can connect to the simulation database:

In [ ]:
citros = CitrosDB(simulation='simulation_lunar_hopper', batch='Lunar_hopper')

citros.info().print()

As you can see in the output above, we've got some information about our simulation run (batch):
1. Size of data
2. Sid information. In case we have more then one sid - this will be shown here.
3. Number and a list of topics. First three topics are build-in system topics, and the last one is our simulation results. 

Let's go further:

In [ ]:
citros.sid([0]).info().print()

This shows us more detailed statistics about topics: time stamps, durations, number of messages etc.
Let's inspect our data topic:

In [ ]:
citros.topic('/lunar_hopper/state').sid(0).info().print()

Here we can see that our data topic contains 99 messages, and each one's type is a Float64MultiArray. 
Let's print all raw data:

In [ ]:
print(citros.topic('/lunar_hopper/state').data())

To get all json-data columns for a specific topic, function data() is used. It returns pandas DataFrame. To get a specific columns, pass their labels in a list as an argument:

In [ ]:
print(citros.topic('/lunar_hopper/state').data(['data.data[0]','data.data[1]'])[['data.data[0]', 'data.data[1]']])

Now we can create some plots. Let's plot coordinates along one of the axes (supposing messages come once a second): 

In [ ]:
fig1, ax1 = plt.subplots()



citros.time_plot(ax1, 

                 topic_name = '/lunar_hopper/state', 

                 var_name = 'data.data[0]', 

                 time_step = 1, 

                 sids = [0,1,2,3,4,5,6,7,8,9], 

                 y_label = 'X coords', title_text = 'X coords vs. Time')

Let's investigate how powerfull the Isp influence is:

In [ ]:
# Defining the list of isp

isp_list = [i for i in range(200,300,10)]

# Setting Dataframe

df = citros.topic('/lunar_hopper/state').set_order({'sid':'asc'}).data('data.data[1]')
sid_list = list(set(df['sid']))
data0_list = []
for s in sid_list:
    id_max = df[df['sid'] == s]['rid'].idxmax()
    data0_list.append(df['data.data[1]'].loc[id_max])

print(data0_list)
fig, ax = plt.subplots()

#Adjusting colors
c = np.random.choice(50, 10, replace=False)
scatter = ax.scatter(isp_list, data0_list,c=c)

# Create legend entries for each point
legend_labels = [str(i) for i in range(10)]

# Initialize a list to store legend handles
legend_handles = []

# Loop through the points and create legend entries with matching colors
for i, label in enumerate(legend_labels):
    color = scatter.to_rgba(c[i])  # Get the color of the corresponding point
    legend_handles.append(plt.Line2D([0], [0], marker='o', color='w', label=label, markerfacecolor=color, markersize=10))

# Add the legend with custom handles
legend1 = ax.legend(handles=legend_handles, loc="upper left", title="sid")
ax.add_artist(legend1)
ax.grid()
ax.set_ylabel('hopping distance, m')
ax.set_xlabel('Initial altitude, m')
ax.set_title('Maximum hopping distance vs Initial altitude')
